In [1]:
"""
   Copyright 2023 Raphaël Isvelin

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""

VIEWER_THEME = "light"  # light, dark
VIEWER_HEIGHT = 900
ENABLE_REPLAY = True

import cadquery as cq

from jupyter_cadquery import show as jcq_show
from jupyter_cadquery import (
    versions,
    PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer, get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
enable_replay(ENABLE_REPLAY)

from cq_warehouse.fastener import (
    polygon_diagonal,
    read_fastener_parameters_from_csv,
    Screw, SocketHeadCapScrew, HeatSetNut, PlainWasher,
    HexNutWithFlange, HexHeadScrew, PanHeadScrew, CounterSunkScrew
)
import cq_warehouse.extensions

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay
Note: To get rid of this warning, use 'enable_replay(False)'


In [2]:
cv = open_viewer("Assembly", anchor="right")  # sets default viewer

objects_to_hide = [
    "/Masks"
]

def hide_objects_matching_strings(strings, keep_edges=True, cv=cv):
    for state in cv.widget.states:
        should_hide = False
        for s in strings:
            if s in state:
                should_hide = True
                break
        if should_hide:
            cv.update_states({
                state: (0, 0 if keep_edges else 0),
            })
            
def show(obj, viewer="Assembly", anchor="right", hide_contains=objects_to_hide):
    cv = jcq_show(
        obj,
        viewer=viewer,
        anchor=anchor,
        #cad_width=1640,
        height=VIEWER_HEIGHT,
        theme=VIEWER_THEME,
        collapse=1,
        optimal_bb=True,
        render_edges=True,
        axes=True,
        axes0=True,
        grid=[True, True, True],
        #black_edges=True,
        reset_camera=False,
        #show_parent=True,
        #timeit=True,
        #js_debug=True
    )
    hide_objects_matching_strings(hide_contains, cv=cv)

def show_b(obj, viewer="Quick", hide_contains=objects_to_hide):
    cv2 = open_viewer(viewer, anchor="split-bottom")
    show(obj, viewer, anchor="split-bottom", hide_contains=hide_contains)

def show_x(obj, hide_contains=objects_to_hide):
    show(obj, viewer=None, hide_contains=hide_contains)

def show_part(part: Part, bottom=False):
    footprint_assembly = (
        cq.Assembly()
            .add(part.debug_objects.footprint.inside, name="Inside", color=cq.Color(1, 0, 1))
            .add(part.debug_objects.footprint.outside, name="Outside", color=cq.Color(0, 1, 1))
    )
    test_assembly = (
        cq.Assembly(None, name="test_assembly")
            .add(part.mask, name="Mask", color=cq.Color(0, 1, 0))
            .add(part.part, name="Part")
            .add(part.assembly_parts_to_cq_assembly(), name="Part assembly")
            .add(footprint_assembly, name="Footprint")
    )
    debug_assembly = cq.Assembly()
    debug_assembly.add(part.debug_objects.hole, name="Hole", color=cq.Color(1, 0, 0))
    for debug_obj_name in part.debug_objects.others.keys():
        obj = part.debug_objects.others[debug_obj_name]
        if not isinstance(obj, dict):
            debug_assembly.add(obj, name=debug_obj_name, color=cq.Color(0.5, 0.7, 0.5))
    test_assembly.add(debug_assembly, name="Debug")

    if bottom: show_b(test_assembly)
    else: show(test_assembly)

## Build test parts

In [3]:
import sys
sys.path.append("../src")

from cq_enclosure_builder import PartFactory as pf
from cq_enclosure_builder import Panel, PanelSize
from cq_enclosure_builder import Enclosure, Face
from cq_enclosure_builder.knobs_and_caps import KNOB_12_6_x_15_8

pf.set_default_types({
    "jack": '6.35mm PJ-612A',
    "usb_a": 'XXX',
    "usb_c": 'XXX',
    "button": 'SPST XX',
    "encoder": 'XXX',
    "screen": 'HDMI 5 inch XXX',
    "potentiometer": 'WH148',
    "banana": 'XXX',
    "barrel_plug": 'XXX',
    "rca": 'XXX',
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2,
    "pot_knob": KNOB_12_6_x_15_8
})

usb_c = pf.build_usb_c()
usb_c_v = pf.build_usb_c(orientation_vertical=True)
usb_a = pf.build_usb_a()
usb_a_v = pf.build_usb_a(orientation_vertical=True)
spst = pf.build_button()
encoder = pf.build_encoder() 
jack_35 = pf.build_jack(part_type='3.5mm XXX')
jack_635 = pf.build_jack()
screen = pf.build_screen()
pot = pf.build_potentiometer()
banana = pf.build_banana()
barrel_plug = pf.build_barrel_plug()
rca = pf.build_rca()


show_part(pot)

VALIDATING CLASS: UsbCXxxPart
VALIDATING CLASS: UsbCXxxPart
VALIDATING CLASS: UsbAXxxPart
VALIDATING CLASS: UsbAXxxPart
VALIDATING CLASS: ButtonSpstXxxPart
VALIDATING CLASS: EncoderXxxPart
VALIDATING CLASS: Jack3_5mmXxxPart
VALIDATING CLASS: Jack6_35mmPj612aPart
TODO Need to move all workplanes so the hole's center is at 0,0
VALIDATING CLASS: Hdmi5InchXxxPart
VALIDATING CLASS: PotentiometerWh148Part
VALIDATING CLASS: BananaXxxPart
VALIDATING CLASS: BarrelPlugXxxPart
VALIDATING CLASS: RcaXxxPart
100% ⋮————————————————————————————————————————————————————————————⋮ (12/12)  0.17s


## Panels etc.

In [4]:
import cadquery as cq
from cq_enclosure_builder import Face
from cq_enclosure_builder.part import Part

class PanelSize:
    def __init__(self, width, length, wall_thickness, total_thickness):
        self.width = width
        self.length = length
        self.wall_thickness = wall_thickness
        self.total_thickness = total_thickness

class Panel:
    def __init__(self,
                 face: Face,
                 top_view_width,
                 top_view_length,
                 wall_thickness,
                 color=None,
                 part_color=None,
                 alpha=1.0):
        self.face = face
        self.size = PanelSize(top_view_width, top_view_length, wall_thickness, wall_thickness)
        self._color = color
        if self._color == None:
            self._color = self.face.default_color
        self._part_color = part_color
        if self._part_color == None:
            self._part_color = self.face.default_part_color
        self.panel = None
        self.mask = self._rotate_to_face(
            cq.Workplane("front")
                .workplane()
                .box(top_view_width, top_view_length, wall_thickness, centered=(True, True, False))
        )
        self.debug_assemblies = {}
        self.debug_assemblies["hole"] = None
        self.debug_assemblies["footprint_in"] = None
        self.debug_assemblies["footprint_out"] = None
        self.debug_assemblies["other"] = None
        self.debug_assemblies["combined"] = cq.Assembly(None, name=self.face.label + " - Debug")
        self._alpha = alpha
        self._parts_to_add = []

    def add(self, label: str, part: Part, rel_pos=None, abs_pos=None):
        print(self.face.label + ": adding part '" + label + "'")
        pos = None
        if rel_pos == None and abs_pos == None:
            raise ValueError("Either rel_pos or abs_pos must be set.")
        elif rel_pos == None:
            pos = (
                abs_pos[0] - self.size.width/2,
                abs_pos[1] - self.size.length/2
            )
        elif abs_pos==None:
            pos = rel_pos
        self._parts_to_add.append({
            "part": part,
            "label": label,
            "pos": pos
        })
        return self

    def assemble(self):
        wall = (
            cq.Workplane("front")
                .workplane()
                .box(self.size.width, self.size.length, self.size.wall_thickness,
                     centered=(True, True, False))
        )
        self.panel = cq.Assembly(None, name="Panel TOP")
        for part_to_add in self._parts_to_add:
            part_obj = part_to_add["part"]
            self._add_part_to_debug_assemblies(part_to_add)
            wall = wall.cut(
                part_obj.mask.translate([*part_to_add["pos"], 0])
            )
            if part_obj.assembly_parts != None:
                self.panel = self.panel.add(
                    self._translate_assembly_objects_and_rotate_to_face(part_obj.assembly_parts, [*part_to_add["pos"], 0]))
                    #self._rotate_assembly_to_face(part_obj.part_assembly.translate([*part_to_add["pos"], 0])))
            else:
                self.panel = self.panel.add(
                    self._rotate_to_face(
                        part_obj.part.translate([*part_to_add["pos"], 0])),
                    name=part_to_add["label"],
                    color=cq.Color(*self._part_color, 1.0)
                )
            if part_obj.size.thickness > self.size.total_thickness:
                self.size.total_thickness = part_obj.size.thickness
        self.panel = self.panel.add(self._rotate_to_face(wall), name="Wall", color=cq.Color(*self._color, self._alpha))
        self.debug_assemblies["combined"] = self._build_combined_debug_assembly()
        return self

    def _rotate_to_face(self, wp):
        if self.face == Face.TOP:
            wp = wp.mirror("XY")
        elif self.face == Face.BOTTOM:
            wp = wp.mirror("XZ")
        elif self.face == Face.BACK:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.mirror("YZ")
        elif self.face == Face.FRONT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.mirror("XZ")
        elif self.face == Face.LEFT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.rotate((0, 0, 0), (0, 0, 1), 90)
            wp = wp.mirror("XZ")
        elif self.face == Face.RIGHT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.rotate((0, 0, 0), (0, 0, 1), -90)
            wp = wp.mirror("XZ")
        return wp

    def _translate_assembly_objects_and_rotate_to_face(self, assembly_parts, translation):
        assembly = cq.Assembly()
        for assembly_part in assembly_parts:
            part, loc, name, color = assembly_part.as_assembly_add_parameters()
            part = part.translate(translation)
            part = self._rotate_to_face(part)
            assembly.add(part, name=name, color=color)
        return assembly

    def _add_part_to_debug_assemblies(self, part_to_add):
        part = part_to_add["part"]
        part_pos = part_to_add["pos"]
        part_label: str = part_to_add["label"]
        if part.debug_objects.hole != None:
            if self.debug_assemblies["hole"] == None:
                self.debug_assemblies["hole"] = cq.Assembly()
            hole = self._rotate_to_face(part.debug_objects.hole.translate([*part_pos, 0]))
            self.debug_assemblies["hole"] = self.debug_assemblies["hole"].add(hole, name=part_label, color=cq.Color(1, 0, 0))
        if part.debug_objects.footprint.inside != None:
            if self.debug_assemblies["footprint_in"] == None:
                self.debug_assemblies["footprint_in"] = cq.Assembly()
            footprint_in = self._rotate_to_face(part.debug_objects.footprint.inside.translate([*part_pos, 0]))
            self.debug_assemblies["footprint_in"] = self.debug_assemblies["footprint_in"].add(footprint_in, name=part_label, color=cq.Color(1, 0, 1))
        if part.debug_objects.footprint.outside != None:
            if self.debug_assemblies["footprint_out"] == None:
                self.debug_assemblies["footprint_out"] = cq.Assembly()
            footprint_out = self._rotate_to_face(part.debug_objects.footprint.outside.translate([*part_pos, 0]))
            self.debug_assemblies["footprint_out"] = self.debug_assemblies["footprint_out"].add(footprint_out, name=part_label, color=cq.Color(0, 1, 1))
        other_debug_assembly = None
        for key in part.debug_objects.others.keys():
            if other_debug_assembly == None:
                other_debug_assembly = cq.Assembly(None)
                self.debug_assemblies["other"] = cq.Assembly()
            debug_part = self._rotate_to_face(part.debug_objects.others[key].translate([*part_pos, 0]))
            other_debug_assembly = other_debug_assembly.add(debug_part, name=key)
        if other_debug_assembly != None:
            if self.debug_assemblies["other"] == None:
                self.debug_assemblies["other"] = cq.Assembly()
            self.debug_assemblies["other"] = self.debug_assemblies["other"].add(other_debug_assembly, name=part_label, color=cq.Color(1, 1, 0))

    def _build_combined_debug_assembly(self):
        combined = self.debug_assemblies["combined"]
        combined = combined.add(self.mask, name="Mask", color=cq.Color(0, 1, 0, 0.5))
        if self.debug_assemblies["hole"] != None:
            combined = combined.add(self.debug_assemblies["hole"], name="Holes")
        if self.debug_assemblies["footprint_in"] != None:
            combined = combined.add(self.debug_assemblies["footprint_in"], name="Footprint in")
        if self.debug_assemblies["footprint_out"] != None:
            combined = combined.add(self.debug_assemblies["footprint_out"], name="Footprint out")
        if self.debug_assemblies["other"] != None:
            combined = combined.add(self.debug_assemblies["other"], name="Other")
        return combined


In [17]:
from enum import Enum
import cq_warehouse.extensions
from cq_enclosure_builder.screws.flat_head_screw import FlatHeadScrew

class TaperOptions(Enum):
    NO_TAPER = 1
    TAPER_ANGLE = 2
    TAPER_SIDE = 3

class DefaultScrewBlock:
    # Hardcoded to one specific Screw class; will refactor when the need comes.
    SCREW_CLASS = FlatHeadScrew
    SCREW_MODEL_NAME = "aliexpress"
    SCREW_SIZE_REFERENCES = {
        "m1": "M1.4-0.3",
        "m2": "M2-0.4",
        "m3": "M3-0.5",
        "m4": "M4-0.7",
        "m5": "M5-0.8",
    }
    BLOCK_SIZES = {
        "m1": ( 5,  5),
        "m2": ( 6,  6),
        "m3": ( 8,  8),
        "m4": (10, 10),
        "m5": (12, 12),
    }

    DEFAULT_FIT = "Loose"
    DEFAULT_COUNTER_SUNK = False
    DEFAULT_SCREW_HOLE_DEPTH = None  # None -> fully go through
    
    @staticmethod
    def generic_screw(screw_size, block_thickness, screw_hole_depth, counter_sunk, fit, taper: TaperOptions, taper_rotation):
        if screw_size not in DefaultScrewBlock.SCREW_SIZE_REFERENCES or screw_size not in DefaultScrewBlock.BLOCK_SIZES:
            raise ValueError("Unknown screw size '" + screw_size + "'")

        screw_size_reference = DefaultScrewBlock.SCREW_SIZE_REFERENCES[screw_size]
        fastener = DefaultScrewBlock.SCREW_CLASS(size=screw_size_reference, fastener_type=DefaultScrewBlock.SCREW_MODEL_NAME, length=20)

        block_size = (*DefaultScrewBlock.BLOCK_SIZES[screw_size], block_thickness)
        if screw_hole_depth == None:
            screw_hole_depth = block_size[2]

        screw_block = (
            cq.Workplane("XY")
                .box(*block_size, centered=(True, True, False))
                .faces(">Z").workplane().pushPoints([(0,0)])
                .threadedHole(fastener=fastener, depth=screw_hole_depth, fit=fit, counterSunk=counter_sunk)
        )
        if taper == TaperOptions.TAPER_ANGLE:
            screw_block.add(
                cq.Workplane("XY")
                    .rect(block_size[0]*2, block_size[1]*2)
                    .extrude(block_size[0]*2, taper=45)
                    .cut(cq.Workplane("XY").rect(block_size[0], block_size[1]).extrude(block_size[0]*2).translate([-(block_size[0]/2), -(block_size[1]/2), 0]))
                    .cut(cq.Workplane("XY").rect(block_size[0], block_size[1]).extrude(block_size[0]*2).translate([-(block_size[0]/2), block_size[1]/2, 0]))
                    .cut(cq.Workplane("XY").rect(block_size[0], block_size[1]).extrude(block_size[0]*2).translate([block_size[0]/2, -(block_size[1]/2), 0]))
                    .translate([-(block_size[0]/2), -(block_size[1]/2), screw_hole_depth])
                    .rotate((0, 0, 0), (0, 0, 1), -taper_rotation)
            )
        elif taper == TaperOptions.TAPER_SIDE:
            screw_block.add(
                cq.Workplane("XZ")
                    .moveTo(0, 0)
                    .lineTo(block_size[0], 0)
                    .lineTo(0, block_size[0])
                    .close()
                    .extrude(block_size[1])
                    .translate([-(block_size[0]/2), block_size[1]/2, screw_hole_depth])
                    .rotate((0, 0, 0), (0, 0, 1), -taper_rotation)
            )

        mask = cq.Workplane("XY").box(*block_size, centered=(True, True, False))
        return {
            "block": screw_block,
            "mask": mask,
            "size": block_size
        }

    @staticmethod
    def m1(block_thickness, screw_hole_depth=DEFAULT_SCREW_HOLE_DEPTH, counter_sunk=DEFAULT_COUNTER_SUNK, fit=DEFAULT_FIT, taper: TaperOptions = TaperOptions.NO_TAPER, taper_rotation = 0):
        return DefaultScrewBlock.generic_screw("m1", block_thickness, screw_hole_depth, counter_sunk, fit, taper, taper_rotation)

    @staticmethod
    def m2(block_thickness, screw_hole_depth=DEFAULT_SCREW_HOLE_DEPTH, counter_sunk=DEFAULT_COUNTER_SUNK, fit=DEFAULT_FIT, taper: TaperOptions = TaperOptions.NO_TAPER, taper_rotation = 0):
        return DefaultScrewBlock.generic_screw("m2", block_thickness, screw_hole_depth, counter_sunk, fit, taper, taper_rotation)

    @staticmethod
    def m3(block_thickness, screw_hole_depth=DEFAULT_SCREW_HOLE_DEPTH, counter_sunk=DEFAULT_COUNTER_SUNK, fit=DEFAULT_FIT, taper: TaperOptions = TaperOptions.NO_TAPER, taper_rotation = 0):
        return DefaultScrewBlock.generic_screw("m3", block_thickness, screw_hole_depth, counter_sunk, fit, taper, taper_rotation)

    @staticmethod
    def m4(block_thickness, screw_hole_depth=DEFAULT_SCREW_HOLE_DEPTH, counter_sunk=DEFAULT_COUNTER_SUNK, fit=DEFAULT_FIT, taper: TaperOptions = TaperOptions.NO_TAPER, taper_rotation = 0):
        return DefaultScrewBlock.generic_screw("m4", block_thickness, screw_hole_depth, counter_sunk, fit, taper, taper_rotation)

    @staticmethod
    def m5(block_thickness, screw_hole_depth=DEFAULT_SCREW_HOLE_DEPTH, counter_sunk=DEFAULT_COUNTER_SUNK, fit=DEFAULT_FIT, taper: TaperOptions = TaperOptions.NO_TAPER, taper_rotation = 0):
        return DefaultScrewBlock.generic_screw("m5", block_thickness, screw_hole_depth, counter_sunk, fit, taper, taper_rotation)

In [18]:
s = DefaultScrewBlock.m5(5, taper=TaperOptions.TAPER_ANGLE, taper_rotation=90)
show(s["block"])  #.add(s["mask"]))

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.13s


In [21]:
def explode(pos_array, walls_explosion_factor=2.0):
    return [x * walls_explosion_factor for x in pos_array]

class EnclosureSize:
    def __init__(self, inner_width, inner_length, inner_thickness, wall_thickness):
        self.inner_width = inner_width
        self.inner_length = inner_length
        self.inner_thickness = inner_thickness
        self.wall_thickness = wall_thickness

class Enclosure:
    def __init__(
        self,
        size: EnclosureSize,
        lid_on_face: Face = Face.BOTTOM,
        lid_panel_size_error_margin = 0.8,  # meaning margin/2 on each side
        no_fillet_top = False,
        no_fillet_bottom = False,
    ):
        super().__init__()

        inner_width = size.inner_width
        inner_length = size.inner_length
        inner_thickness = size.inner_thickness
        wall_thickness = size.wall_thickness
        self.size = size
        self.no_fillet_top = no_fillet_top
        self.no_fillet_bottom = no_fillet_bottom

        if lid_on_face != Face.BOTTOM:
            # TODO: add support (upddate the correct face in panels_specs)
            # TODO nice-to-have: also add support for multiple lids (faces)
            raise ValueError("lid_on_face: only value supported for now is BOTTOM, got " + str(lid_on_face))

        self.frame = (
            cq.Workplane("front")
                .box(inner_width - 4, inner_length - 4, inner_thickness - 4, centered=(True, True, False))
                #.faces("+Z")
                .shell(wall_thickness)
                #.translate([2, 2, 2])
        )
        self.panels_specs = [
            (Face.TOP,    (inner_width-4,  inner_length-4,    wall_thickness),  [0, 0, inner_thickness - wall_thickness],   0.9 ),
            (Face.BOTTOM, (inner_width-4-lid_panel_size_error_margin, inner_length-4-lid_panel_size_error_margin, wall_thickness),  [0, 0, -wall_thickness],    0.9 ),
            (Face.FRONT,  (inner_width-4,  inner_thickness-4, wall_thickness),  [0, -(inner_length/2), inner_thickness/2 - wall_thickness], 0.9 ),
            (Face.BACK,   (inner_width-4,  inner_thickness-4, wall_thickness),  [0, inner_length/2, inner_thickness/2 - wall_thickness],  0.9 ),
            (Face.LEFT,   (inner_length-4, inner_thickness-4, wall_thickness),  [-(inner_width/2), 0, inner_thickness/2 - wall_thickness], 0.9 ),
            (Face.RIGHT,  (inner_length-4, inner_thickness-4, wall_thickness),  [inner_width/2, 0, inner_thickness/2 - wall_thickness],  0.9 ),
        ]

        self.panels = {}
        self.screws_specs = []
        self.scews = []

        for info in self.panels_specs:
            self.panels[info[0]] = Panel(info[0], *info[1], alpha=info[3])

    def add_part_to_face(self, face: Face, part_label: str, part: Part, rel_pos=None, abs_pos=None):
        self.panels[face].add(part_label, part, rel_pos, abs_pos)
        return self

    def add_screw_for_lid_panel(self, size: str = "m3", rel_pos = None, abs_pos = None):
        # TODO add support abs_pos and validation
        pass

    def assemble(self, walls_explosion_factor=1.0, lid_panel_shift=0.0):
        for panel in self.panels.values():
            panel.assemble()

        panels_assembly, panels_masks_assembly = self._build_panels_assembly(walls_explosion_factor, lid_panel_shift)
        frame_assembly = self._build_frame_assembly(panels_masks_assembly)

        footprints_assembly = self._build_debug_assembly([("footprint_in", "I"), ("footprint_out", "O")], walls_explosion_factor, lid_panel_shift)
        holes_assembly = self._build_debug_assembly([("hole", "")], walls_explosion_factor, lid_panel_shift)
        other_debug_assembly = self._build_debug_assembly([("other", "")], walls_explosion_factor, lid_panel_shift)

        self.debug = (
            cq.Assembly(None, name="Box")
                .add(footprints_assembly, name="Footprints")
                .add(holes_assembly, name="Holes")
                .add(other_debug_assembly, name="Others")
                .add(panels_masks_assembly, name="Panels masks")
        )
        self.assembly = (
            cq.Assembly(None, name="Box")
                .add(panels_assembly, name="Panels")
                .add(frame_assembly, name="Frame")
                .add(self.debug, name="Debug")
        )
        return self

    def _get_debug(self, panel: Panel, assembly_name="combined"):
        if assembly_name in panel.debug_assemblies:
            return panel.debug_assemblies[assembly_name]
        return None

    def _build_panels_assembly(self, walls_explosion_factor, lid_panel_shift):
        a = cq.Assembly(None)
        masks_a = cq.Assembly(None)
        for face, size, position, alpha in self.panels_specs:
            panel: Panel = self.panels[face]
            translated_mask = panel.mask.translate(position)
            masks_a.add(translated_mask, name=(face.label + " mask"), color=cq.Color(0, 1, 0))
            if face == Face.BOTTOM:
                position = (position[0], position[1], position[2] - lid_panel_shift)
            translated_panel = panel.panel.translate(explode(position, walls_explosion_factor))
            a.add(translated_panel, name=face.label)
        return (a, masks_a)

    def _build_frame_assembly(self, panels_masks_assembly) -> cq.Workplane:
        """
        Get the frame of the enclosure (the masks of the panels will be .cut during the assembly).
        In this implemenmtation, it's only possible to remove the fillet
        on the top and bottom of the enclosure (TODO: better implementation needed when lid_on_face != Face.BOTTOM allowed),
        but you can to override the method if you need something more custom (and feel free to contribute ;) )
        """
        # TODO: when implementing the lid on other faces that BOTTOM, will have to remake that code better

        wall_thickness = self.size.wall_thickness

        shell_faces_filter = []
        shell_size = (
            self.size.inner_width - wall_thickness*2,
            self.size.inner_length - wall_thickness*2,
            self.size.inner_thickness - wall_thickness*2
        )
        shell_translate_z = 0

        if self.no_fillet_top:
            shell_faces_filter.append("+Z")
            shell_size[2] = shell_box_thickness + wall_thickness
            shell_translate_z = shell_translate_z + wall_thickness/2
        if self.no_fillet_bottom:
            shell_faces_filter.append("-Z")
            shell_size[2] = shell_box_thickness + wall_thickness
            shell_translate_z = shell_translate_z - wall_thickness/2

        no_faces_filter = "+Z and -Z"  # as two faces cannot be true at once--cheeky way to avoid `.faces("")`
        shell_faces = no_faces_filter if len(shell_faces_filter) == 0 else " or ".join(shell_faces_filter)

        shell = (
            cq.Workplane()
                .box(*shell_size, centered=(True, True, False))
                .faces(shell_faces)
                .shell(wall_thickness)
                .translate([0, 0, shell_translate_z])
        )

        return shell.cut(panels_masks_assembly.toCompound())  #self._cut_panels_masks_from_frame(shell)

    def _cut_panels_masks_from_frame(self, frame: cq.Workplane) -> cq.Workplane:
        for face, size, position, alpha in self.panels_specs:
            panel: Panel = self.panels[face]
            translated_mask = panel.mask.translate(position)
            frame = frame.cut(translated_mask)
        return frame

    def _build_debug_assembly(self, assemblies_specs, walls_explosion_factor, lid_panel_shift):
        a = cq.Assembly(None)
        for face, size, position, alpha in self.panels_specs:
            if face == Face.BOTTOM:
                position = (position[0], position[1], position[2] - lid_panel_shift)
            panel: Panel = self.panels[face]
            for assembly_type, assembly_name_suffix in assemblies_specs:
                debug_assembly = self._get_debug(panel, assembly_type)
                if debug_assembly != None:
                    translated_debug = debug_assembly.translate(explode(position, walls_explosion_factor))
                    a.add(translated_debug, name=(f"{face.label} {assembly_name_suffix}"))
        return a

In [24]:
ENCLOSURE_INNER_WIDTH =  220
ENCLOSURE_INNER_LENGTH = 150
ENCLOSURE_INNER_THICKNESS = 38
ENCLOSURE_WALLS_THICKNESS = 2

enclosure_size = EnclosureSize(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_INNER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
enclosure = Enclosure(size=enclosure_size)
# enclosure.add_part_to_face(Face.TOP, "Screen", pf.build_screen(), rel_pos=(0, -4))
enclosure.add_part_to_face(Face.TOP, "SPST 1", pf.build_button(), abs_pos=(20, 30))
enclosure.add_part_to_face(Face.TOP, "SPST 2", pf.build_button(), abs_pos=(20, 100))
enclosure.add_part_to_face(Face.TOP, "SPST 3", pf.build_button(), abs_pos=(200, 30))
enclosure.add_part_to_face(Face.TOP, "Encoder", pf.build_encoder(), abs_pos=(200, 100))
for i in range(0, 4):
    enclosure.add_part_to_face(Face.BACK, f"Js{i}", pf.build_jack(part_type='3.5mm XXX'), abs_pos=(8, 4.5 + i*9))
for i in range(0, 7):
    enclosure.add_part_to_face(Face.BACK, f"Jb{i}", pf.build_jack(), abs_pos=(24 + i*18, 10))
    enclosure.add_part_to_face(Face.BACK, f"Jt{i}", pf.build_jack(), abs_pos=(24 + i*18, 18+9))
# enclosure.add_part_to_face(Face.LEFT, "USB", pf.build_usb_a(), rel_pos=(0, 0))
# enclosure.add_part_to_face(Face.LEFT, "USB V", pf.build_usb_a(orientation_vertical=True), rel_pos=(40, 0))
# enclosure.add_part_to_face(Face.RIGHT, "USB C", pf.build_usb_c(), rel_pos=(0, 0))
# enclosure.add_part_to_face(Face.RIGHT, "USB C V", pf.build_usb_c(orientation_vertical=True), rel_pos=(40, 0))

enclosure.assemble(walls_explosion_factor=1.0, lid_panel_shift=100)

show(enclosure.assembly, hide_contains=["Masks", "Debug"])

VALIDATING CLASS: ButtonSpstXxxPart
TOP: adding part 'SPST 1'
VALIDATING CLASS: ButtonSpstXxxPart
TOP: adding part 'SPST 2'
VALIDATING CLASS: ButtonSpstXxxPart
TOP: adding part 'SPST 3'
VALIDATING CLASS: EncoderXxxPart
TOP: adding part 'Encoder'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js0'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js1'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js2'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js3'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb0'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt0'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb1'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt1'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb2'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt2'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb3'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt3'
VALIDATING CLA

In [23]:
no_fillet_top = False
no_fillet_bottom = False

enclosure_wall_thickness = 2

# TODO: when implementing the lid on other faces that BOTTOM, will have to remake that code better

shell_faces_filter = []
shell_box_thickness = 10
shell_translate_z = 0

if no_fillet_top:
    shell_faces_filter.append("+Z")
    shell_box_thickness = shell_box_thickness + enclosure_wall_thickness
    shell_translate_z = shell_translate_z + enclosure_wall_thickness/2
if no_fillet_bottom:
    shell_faces_filter.append("-Z")
    shell_box_thickness = shell_box_thickness + enclosure_wall_thickness
    shell_translate_z = shell_translate_z - enclosure_wall_thickness/2


no_faces_filter = "+Z and -Z"  # as two faces cannot be true at once
shell_faces = no_faces_filter if len(shell_faces_filter) == 0 else " or ".join(shell_faces_filter)

shell = (
    cq.Workplane()
        .box(10, 10, shell_box_thickness)
        .faces(shell_faces)
        .shell(enclosure_wall_thickness)
        .translate([0, 0, shell_translate_z])
)

show(shell)